In [2]:
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
import pandas as pd


In [3]:

geom_file="/glade/work/afox/jedi/Jedi_experiments/input/wrf_hydro_nwm_files/conus/geometry_cfs_long_range_ens.nc"
ncfile = netCDF4.Dataset(geom_file, mode='r')

# Get metadata from the file
npx = ncfile.dimensions["west_east"].size
npy = ncfile.dimensions["south_north"].size
npz = 1

lons_conus = ncfile.variables["XLONG"][:]
lats_conus = ncfile.variables["XLAT"][:]

# Print field dimensions
print(" Grid dimensions", npx, 'x', npy, 'x', npz)

# Zero out array to fill with field
hgt = np.zeros((npy, npx))

hgt[:,:] = ncfile.variables["HGT"][:,:]

 Grid dimensions 4608 x 3840 x 1


In [5]:
make_obs_file = '/glade/work/afox/jedi/whj_test/build/wrf_hydro_nwm_jedi/test/Data/make_obs/make_obs_3d.snow_depth_0000.nc'
ncfile = netCDF4.Dataset(make_obs_file, mode='r')
lons = ncfile['/MetaData/longitude'][:]
lats = ncfile['/MetaData/latitude'][:]
obs_err = ncfile['/ObsError/snow_depth'][:]
obs_val = ncfile['/ObsValue/snow_depth'][:]


In [6]:
obs_file = '/glade/u/home/afox/work/jedi/Jedi_experiments/ctest_letkf_oi_obs_depth.csv'
df = pd.read_csv(obs_file)
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
1,99998,XXXX2,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
2,99997,XXXX3,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
3,99996,XXXX4,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
4,99995,XXXX5,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
5,99994,XXXX6,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
6,99993,XXXX7,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
7,99992,XXXX8,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
8,99991,XXXX9,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m
9,99990,XXXX10,9.999,0.04,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_depth_m


In [7]:
df['ObsValue']=obs_val
df['ObsError']=obs_err
df['longitude']=lons
df['latitude']=lats

In [8]:
latitude = df['latitude']
longitude = df['longitude']

nobs = np.shape(longitude)
nobs = nobs[0]

In [9]:
lon_req = longitude
lat_req = latitude

array1 = np.asarray(lons_conus)
array2 = np.asarray(lats_conus)

height_out = np.zeros(nobs)

for i in range(nobs):
    lons_diff = (array1 - (lon_req[i]))**2
    lats_diff = (array2 - (lat_req[i]))**2
    comb_diff = lons_diff+lats_diff
    idx = comb_diff.argmin()
    height_out[i] = hgt.flatten()[idx]

In [10]:
df['dem_elev_m']=height_out
df['rec_elev_m']=height_out
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,0.408602,0.04,0,2881.344727,2881.344727,38.861473,-106.554100,2017-01-01T00:00:00Z,snow_depth_m
1,99998,XXXX2,0.392698,0.04,0,3472.481445,3472.481445,39.011429,-106.675011,2017-01-01T00:00:00Z,snow_depth_m
2,99997,XXXX3,0.384901,0.04,0,2972.729248,2972.729248,38.911392,-106.586258,2017-01-01T00:00:00Z,snow_depth_m
3,99996,XXXX4,0.510124,0.04,0,3714.192139,3714.192139,39.015091,-106.656250,2017-01-01T00:00:00Z,snow_depth_m
4,99995,XXXX5,0.549557,0.04,0,3310.116699,3310.116699,38.906120,-106.728134,2017-01-01T00:00:00Z,snow_depth_m
5,99994,XXXX6,0.562848,0.04,0,3593.670410,3593.670410,39.040169,-106.713333,2017-01-01T00:00:00Z,snow_depth_m
6,99993,XXXX7,0.453180,0.04,0,3123.326660,3123.326660,38.943928,-106.644928,2017-01-01T00:00:00Z,snow_depth_m
7,99992,XXXX8,0.584000,0.04,0,3466.655762,3466.655762,39.009411,-106.731529,2017-01-01T00:00:00Z,snow_depth_m
8,99991,XXXX9,0.367054,0.04,0,2985.754395,2985.754395,38.943085,-106.606331,2017-01-01T00:00:00Z,snow_depth_m
9,99990,XXXX10,0.532127,0.04,0,3512.529541,3512.529541,38.904781,-106.691795,2017-01-01T00:00:00Z,snow_depth_m


In [ ]:
df.to_csv (obs_file, index = False, header=True)